In [1]:
from utils import *
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate 
from prompt_about_resume_information import create_skills_prompt
import json

- UP CV (file pdf)

In [2]:
import fitz
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text("text")
        if page_num > 0 :
            text += "\n"
    return text

In [4]:
def extract_skill_from_pdf(path_pdf):
    text = extract_text_from_pdf(path_pdf)
    prompt = create_skills_prompt(text=text)
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=1)
    answer = llm.invoke(prompt)
    return answer

In [5]:
answer = extract_skill_from_pdf("ResumesPDF/cv (5).pdf")

d:\LLM for ChatBot\CourseAdvisor\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
data_json = json.loads(answer.content)

In [7]:
# Duyệt và in thông tin
for key, value in data_json.items():
    if isinstance(value, list):
        print(f"{key}:")
        for item in value:
            for inner_key, inner_value in item.items():
                print(f"  {inner_key}: {inner_value}")
    elif isinstance(value, dict):
        print(f"{key}:")
        for inner_key, inner_value in value.items():
            print(f"  {inner_key}: {inner_value}")
    else:
        print(f"{key}: {value}")

Skills:
  TechnicalSkills: [{'Skill': 'Software Engineer', 'Level': 'Intermediate'}, {'Skill': 'Data Analyst', 'Level': 'Intermediate'}, {'Skill': 'Machine Learning', 'Level': 'Intermediate'}, {'Skill': 'Text Analytics', 'Level': 'Intermediate'}, {'Skill': 'Software Development', 'Level': 'Intermediate'}, {'Skill': 'Object Oriented Programming', 'Level': 'Intermediate'}, {'Skill': 'Pandas', 'Level': 'Intermediate'}, {'Skill': 'Numpy', 'Level': 'Intermediate'}, {'Skill': 'Java', 'Level': 'Beginner'}, {'Skill': 'Python', 'Level': 'Intermediate'}, {'Skill': 'SpringBoot', 'Level': 'Intermediate'}, {'Skill': 'Laravel', 'Level': 'Beginner'}]
  SoftSkills: ['Motivated to learn', 'Grow and excel']


### Passing a Chroma Client into Langchain
- Load Vector Database 
    - Udemy
    - Coursera
    - skillshare
    - edx

In [9]:
db_udemy = create_langchain_chroma(db_path="vector_databases/udemy",collection_name="udemy")
# db_coursera = create_langchain_chroma(db_path="vector_databases/coursera",collection_name="coursera")
# db_skillshare = create_langchain_chroma(db_path="vector_databases/skillshare",collection_name="skillshare")
# db_edx = create_langchain_chroma(db_path="vector_databases/edx",collection_name="edx")

In [10]:
def get_relevant_passage(query, db, n_results):
    passage = db.similarity_search_with_score(query, k= n_results)
    return passage
query = "Machine Learning"
results_udemy = get_relevant_passage(query=query, db=db_udemy, n_results= 3)

In [12]:
# for rs in results_udemy:
#     print("_"*20)
#     print(f'titel : {rs[0].page_content}')
#     for key, value in rs[0].metadata.items():
#         print(f"{key} : {value}")

- self Querying

In [ ]:
# 'description' : row['description'],
# 'instructor' : row['instructor'],
# 'rating' : row['rating'],
# 'reviewcount' : row['reviewcount'],
# 'duration' : row['duration'],
# 'lectures' : row['lectures'],
# 'level' : row['level'],

In [18]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="description",
        description="The description of the course",
        type="string",
    ),
    AttributeInfo(
        name="instructor",
        description="The name of the instructor",
        type="string",
    ),
    AttributeInfo(
        name="rating",
        description="The course rating on a scale of 1-5",
        type="float",
    ),
    AttributeInfo(
        name="reviewcount",
        description="The number of reviews for the course",
        type="integer",
    ),
    AttributeInfo(
        name="duration",
        description="The duration of the course in hours",
        type="string",
    ),
    AttributeInfo(
        name="lectures",
        description="The number of lectures in the course",
        type="integer",
    ),
    AttributeInfo(
        name="level",
        description="The level of the course. One of ['All Level', 'Beginner', 'Intermediate', 'Expert']",
        type="string",
    ),
]

document_content_description = "Search and retrieve courses based on user-defined criteria."

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    db_udemy,
    document_content_description,
    metadata_field_info,
    enable_limit=True
)


In [19]:
# This example only specifies a filter
test_rs = retriever.invoke("Search for courses related to Data Analyst at the Intermediate level.")


In [20]:
test_rs

[Document(page_content='The Complete Data Analytics Course in Excel', metadata={'description': 'A complete data analysis course using the tools you already have.', 'duration': '15 total hours', 'instructor': 'Michael Parent', 'lectures': '71 lectures', 'level': 'Intermediate', 'rating': 4.5, 'reviewcount': 768}),
 Document(page_content='Certified Cyber Threat Intelligence Analyst', metadata={'description': 'Learn to intelligently detect and take down cyber threats', 'duration': '9.5 total hours', 'instructor': 'CyberTraining 365', 'lectures': '69 lectures', 'level': 'Intermediate', 'rating': 3.9, 'reviewcount': 861}),
 Document(page_content='Essential Excel for Business Analysts and Consultants', metadata={'description': 'Master the formulas and tricks that will save you time and make you as efficient as consultants in McKinsey, BCG, Bain', 'duration': '5.5 total hours', 'instructor': 'Asen Gyczew', 'lectures': '108 lectures', 'level': 'Intermediate', 'rating': 4.5, 'reviewcount': 941}

In [43]:
retriever.invoke("I want to find artificial intelligence courses rated higher than 4.0")

[Document(page_content='Artificial Intelligence: Advanced Machine Learning', metadata={'description': 'Learn all the advanced skills you need to perform various real-world machine learning tasks in different environments.', 'duration': '3.5 total hours', 'instructor': 'Eduero Academy, Inc.', 'lectures': '47 lectures', 'level': 'Intermediate', 'rating': 4.1, 'reviewcount': 37}),
 Document(page_content='Artificial Intelligence and Machine Learning Made Simple', metadata={'description': 'A non-technical explanation of all the buzzwords around Artificial Intelligence, Machine Learning and Deep Learning.', 'duration': '44 total mins', 'instructor': 'Sertac Ozker', 'lectures': '11 lectures', 'level': 'All Levels', 'rating': 4.3, 'reviewcount': 126}),
 Document(page_content='Artificial Intelligence and Predictive Analysis', metadata={'description': 'This course is a comprehensive understanding of AI concepts and its application using Python and iPython.', 'duration': '6.5 total hours', 'instr

In [44]:
retriever.invoke("I want to find a course on artificial intelligence that has more than 100 views.")

[Document(page_content='Artificial Intelligence and Machine Learning Made Simple', metadata={'description': 'A non-technical explanation of all the buzzwords around Artificial Intelligence, Machine Learning and Deep Learning.', 'duration': '44 total mins', 'instructor': 'Sertac Ozker', 'lectures': '11 lectures', 'level': 'All Levels', 'rating': 4.3, 'reviewcount': 126}),
 Document(page_content='Artificial Intelligence and Predictive Analysis', metadata={'description': 'This course is a comprehensive understanding of AI concepts and its application using Python and iPython.', 'duration': '6.5 total hours', 'instructor': 'EDUCBA Bridging the Gap', 'lectures': '59 lectures', 'level': 'All Levels', 'rating': 4.3, 'reviewcount': 183}),
 Document(page_content='How to use Artificial Intelligence - A guide for everyone!', metadata={'description': 'Benefit today from the technology of tomorrow : Start using Artificial Intelligence to boost your productivity!', 'duration': '1.5 total hours', 'in

In [46]:
retriever.invoke("I want to find a course with Maximilian Schwarzmüller as the instructor.")

[Document(page_content='Accelerated JavaScript Training', metadata={'description': 'Take a deep dive into JavaScript, a Must-Know Language in Web Development, with a Hands-on, Example-driven Approach', 'duration': '7.5 total hours', 'instructor': 'Maximilian Schwarzmüller', 'lectures': '128 lectures', 'level': 'All Levels', 'rating': 4.6, 'reviewcount': 6365}),
 Document(page_content='Accelerated ES6 JavaScript Training', metadata={'description': 'Learn and Use the Future of JavaScript - Today!', 'duration': '6 total hours', 'instructor': 'Maximilian Schwarzmüller', 'lectures': '115 lectures', 'level': 'Intermediate', 'rating': 4.6, 'reviewcount': 5680}),
 Document(page_content='Angular - The Complete Guide (2023 Edition)', metadata={'description': 'Master Angular (formerly "Angular 2") and build awesome, reactive web apps with the successor of Angular.js', 'duration': '36.5 total hours', 'instructor': 'Maximilian Schwarzmüller', 'lectures': '503 lectures', 'level': 'All Levels', 'rati

- Create prompts

In [47]:
def course_search_prompt(question):
    prompt = PromptTemplate(
        input_variables=["question"],
        template="""
            Please search for online courses on relevant websites related to the following content:
            Content: {question}
            The answer should be returned as a JSON object in the following format:
            {{
                "title": "string",
                "description": "string",
                "link": "string"
            }}
            The returned result should be in JSON format without anything added.
            """
    )
    return prompt.format(question=question)


In [5]:
# print(course_search_prompt("English courses for working people"))

In [30]:
def detailed_course_prompt(question):
    prompt = PromptTemplate(
        input_variables=["question"],
        template="""
        Please find the course with the following title: {question}
        The answer should be returned as a JSON object in the following format:
        {{
            "title": "String",
            "Course_Content": "String",
            "Instructor": "String",
            "level": "String",
            "Rating": "String",
            "Duration": "String",
            "cost": "String",
            "Certificate": "String" (if applicable),
            "Prerequisites": "String" (if applicable),
            "link": "String"
        }}
        The returned result should be in JSON format without anything added.
        """
    )
    return prompt.format(question=question)

# Ví dụ sử dụng
question = "The Complete Web Developer Course 3.0"
result = detailed_course_prompt(question)


In [1]:
def generate_answer_prompt(prompt):
    llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=1)
    answer = llm.invoke(prompt)
    return answer

In [32]:
rs = generate_answer_prompt(result)

In [33]:
print(rs.content)

{
"title": "The Complete Web Developer Course 3.0",
"Course_Content": "HTML, CSS, JavaScript, React, Node.js, MongoDB, Python, Django, Flask",
"Instructor": "Rob Percival, Colt Steele",
"level": "Beginner to Advanced",
"Rating": "4.7/5",
"Duration": "Over 60 hours of video content, 110+ downloadable resources, and 43 coding exercises",
"cost": "$199",
"Certificate": "Yes",
"Prerequisites": "No prior programming experience is required",
"link": "https://www.udemy.com/course/the-complete-web-developer-course-2/"
}


In [3]:
rs = generate_answer_prompt("đọc trang web này xem https://www.edx.org/learn/excel/ibm-analyzing-data-with-excel hãy liệt kê nội dung trong trang web")

d:\LLM for ChatBot\CourseAdvisor\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(rs.content)

**Trình độ: Tất cả trình độ**

**Đại học: IBM**

**Chủ đề:** Khoa học dữ liệu

**Mô tả:**

Khóa học trực tuyến Analyzing Data with Excel do IBM cung cấp trên nền tảng edX này sẽ giúp bạn học cách phân tích dữ liệu trong Microsoft Excel, một phần mềm bảng tính phổ biến.

**Nội dung:**

* **Module 1: Giới thiệu về phân tích dữ liệu trong Excel**
    * Hiểu vai trò của Excel trong phân tích dữ liệu
    * Nhập và chuẩn bị dữ liệu trong Excel

* **Module 2: Phân tích dữ liệu định lượng**
    * Thống kê tóm tắt
    * Biểu đồ phân phối tần suất và các biện pháp tản mác
    * Kiểm tra giả thuyết cho dữ liệu định lượng

* **Module 3: Phân tích dữ liệu định tính**
    * Table động và biểu đồ
    * Các công cụ tóm tắt dữ liệu và thao tác văn bản
    * Phân tích nội dung và phân loại

* **Module 4: Trực quan hóa dữ liệu**
    * Tạo các loại biểu đồ khác nhau
    * Hiểu các khái niệm về thiết kế biểu đồ
    * Kể câu chuyện bằng hình ảnh

* **Module 5: Làm việc với dữ liệu lớn trong Excel**
    * Sử